In [28]:
import torch

In [29]:
import torchvision

In [30]:
from comet_ml import Experiment, Optimizer

In [31]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from collections import defaultdict
from tqdm import trange
import torch
import seaborn as sns
import os
import matplotlib.pyplot as plt
from io import BytesIO
import base64
import sklearn
import time
import datetime

plt.style.use("seaborn")
torch.set_default_dtype(torch.float32)

In [32]:
import datetime


In [33]:
import node

In [34]:
from sklearn.metrics import roc_auc_score, f1_score, precision_score, recall_score, balanced_accuracy_score

In [35]:
from tabular_hypernet import HypernetworkPCA, TrainingModes, Hypernetwork
from tabular_hypernet.modules import SimpleNetwork
from tabular_hypernet.training_utils import train_slow_step, train_model, train_carthesian
from tabular_hypernet.interfaces import HypernetworkSklearnInterface, SimpleSklearnInterface
# from ipynb.fs.defs.MNIST_benchmark import test_model
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

In [36]:
from sklearn.metrics import classification_report, balanced_accuracy_score

In [37]:
import imblearn

In [38]:
import pyhopper

In [39]:
from loguru import logger

logger.add("log_Lymf.txt", format='{time:YYYY-MM-DD HH:mm:ss.SSS} | {message}')

3

In [40]:
import seaborn as sns

In [41]:
DEVICE="cuda:0"

In [42]:
from loguru import logger

logger.add("log.txt", format='{time:YYYY-MM-DD HH:mm:ss.SSS} | {message}')

4

In [43]:
GS_METRIC = "balanced_accuracy"

In [44]:
import openml
from openml.datasets import get_dataset

# Load data

In [45]:
DATA = "devanagari"

In [46]:
if DATA == "devanagari":
    transforms = torchvision.transforms.Compose(
    [
        torchvision.transforms.ToTensor(), 
        torchvision.transforms.Grayscale(),
        torchvision.transforms.Lambda(lambda x: torch.flatten(x))
    ]
    )

    consonants = torchvision.datasets.ImageFolder('./Devanagari-kaggle/nhcd/nhcd/consonants', 
                                                  transform=transforms
                                                 )

    vowels = torchvision.datasets.ImageFolder('./Devanagari-kaggle/nhcd/nhcd/vowels', 
                                                  transform=transforms
                                                 )
    
    numerals = torchvision.datasets.ImageFolder('./Devanagari-kaggle/nhcd/nhcd/numerals', 
                                                  transform=transforms
                                                 )
    
    consonants_X = np.asarray([consonants[i][0].numpy() for i in range(len(consonants))]).astype(float)
    consonants_y = np.asarray([consonants[i][1] for i in range(len(consonants))])
    
    vowels_X = np.asarray([vowels[i][0].numpy() for i in range(len(vowels))]).astype(float)
    vowels_y = np.asarray([vowels[i][1] for i in range(len(vowels))]) + consonants_y.max() + 1
    
    numerals_X = np.asarray([numerals[i][0].numpy() for i in range(len(numerals))]).astype(float)
    numerals_y = np.asarray([numerals[i][1] for i in range(len(numerals))]) + vowels_y.max() + 1

    X = np.concatenate((consonants_X, vowels_X, numerals_X), axis=0)
    y = np.concatenate((consonants_y, vowels_y, numerals_y), axis=0)

    
print(X.shape, len(np.unique(y)))

(12912, 784) 58


In [47]:
n_classes = len(np.unique(y if not isinstance(y, tuple) else y_train))
n_classes

58

In [48]:
unique, counts = np.unique(y if not isinstance(y, tuple) else y_train, return_counts=True)

print(dict(zip(unique, counts)))

{0: 205, 1: 205, 2: 205, 3: 205, 4: 205, 5: 205, 6: 205, 7: 205, 8: 205, 9: 205, 10: 205, 11: 205, 12: 205, 13: 205, 14: 205, 15: 205, 16: 205, 17: 205, 18: 205, 19: 205, 20: 205, 21: 205, 22: 205, 23: 205, 24: 205, 25: 205, 26: 205, 27: 205, 28: 205, 29: 205, 30: 205, 31: 205, 32: 205, 33: 205, 34: 205, 35: 205, 36: 221, 37: 221, 38: 221, 39: 221, 40: 221, 41: 221, 42: 221, 43: 221, 44: 221, 45: 221, 46: 221, 47: 221, 48: 288, 49: 288, 50: 288, 51: 288, 52: 288, 53: 288, 54: 288, 55: 288, 56: 288, 57: 288}


In [49]:
n_features = X.shape[1] if not isinstance(X, tuple) else X_train.shape[1]
n_features

784

In [50]:
max_size = int(len(X)*0.7) if not isinstance(X, tuple) else len(X_train)

In [51]:
max_size

9038

In [52]:
GS_DATA_SIZE = max_size // 2
GS_DATA_SIZE

4519

In [53]:
# print('dataset size:', len(X), '|', 'max training size:', max_size)

In [54]:
TEST_RUN = False

# Preprocess

In [55]:
np.random.seed(42)

def prepare_data(X, y, size=None):
    if isinstance(X, tuple) and isinstance(y, tuple):
        X_train, X_test = X
        y_train, y_test = y
        
        if size:
            X_train, _, y_train, _ = train_test_split(X_train, y_train, train_size=size, stratify=y_train)
            
    else:    
        X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=size, stratify=y)
    # X_train, y_train = imblearn.over_sampling.RandomOverSampler(random_state=42).fit_resample(X_train, y_train)
    
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
    
    X_train, X_test, y_train, y_test = [torch.from_numpy(x) for x in [X_train, X_test, y_train, y_test]]
    
    return X_train, X_test, y_train, y_test

In [56]:
X_train, X_test, y_train, y_test = prepare_data(X, y, size=max_size)

## Benchmark

In [57]:
criterion = torch.nn.CrossEntropyLoss()

In [58]:
def _summarize_results(y_pred, y_score, y_test, labels):
    results = []
    for idx, label in enumerate(labels):
        y_pred_filt = y_pred[y_test==idx]
        y_test_filt = y_test[y_test==idx]
        acc = (y_pred_filt==y_test_filt.numpy()).sum()/len(y_test_filt)*100
        results.append({
            "Class": label,
            "Metric": acc
        })
        
    acc = (y_pred==y_test.numpy()).sum()/len(y_test)*100    
    results.append({
        "Class": "Total",
        "Metric": acc
    })
    
    
    results.append({
        "Class": "balanced_accuracy",
        "Metric": balanced_accuracy_score(y_test, torch.from_numpy(y_pred)).item()*100
    })
    
    try:
        results.append({
            "Class": "F1 score",
            "Metric": f1_score(y_test, torch.from_numpy(y_pred)).item()*100
        })
        results.append({
            "Class": "roc_auc",
            "Metric": roc_auc_score(y_test, torch.from_numpy(y_score[:, 1])).item()*100
        })
        results.append({
            "Class": "Precision",
            "Metric": precision_score(y_test, torch.from_numpy(y_pred)).item()*100
        })
        results.append({
            "Class": "Recall",
            "Metric": recall_score(y_test, torch.from_numpy(y_pred)).item()*100
        })
    except ValueError:
        pass
    return results

def test_model(model_fn, data, train_size, label_encoder=None, iters=10, as_numpy=False):
    if TEST_RUN:
        iters = 1
        
    if label_encoder is not None:
        labels = label_encoder.classes_
    else:
        labels = sorted(pd.unique(data[1][0] if isinstance(data[1], tuple) else data[1]))
    
    results = []

    for i in range(iters):
        print('iter', i+1, 'of', iters)
        X_train, X_test, y_train, y_test = prepare_data(*data, train_size)
        
        model = model_fn()

        if as_numpy:
            model.fit(X_train.numpy(), y_train.numpy());
        else:
            model.fit(X_train, y_train)
        
        y_pred = model.predict(X_test)
        y_score = model.predict_proba(X_test)
        results.extend(_summarize_results(y_pred, y_score, y_test, labels))
        
        del model
        torch.cuda.empty_cache()

    dframe = pd.DataFrame.from_dict(results)
#     sns.violinplot(data=dframe[dframe["Class"]!="Loss"], y="Class", x="Metric", orient='h')
    return dframe

### Param search

In [59]:
def pyhopper_best_params(model_fn, param_grid, data_size=GS_DATA_SIZE, metric=GS_METRIC, time="30m", default_params={}, n_jobs=1):
    if TEST_RUN:
        time = 30
        if 'epochs' in param_grid:
            param_grid["epochs"] = pyhopper.choice([10])
        
    logger.info(f"pyhopper_best_params {model_fn.__name__}")
    
    def objective(params):
        torch.cuda.empty_cache()

        model_results = test_model(
                        model_fn(
                            **default_params,
                            **params
                        ),
                        (X, y),
                        data_size,
                        None, 5)
        with open(f"{DATA}_{model_fn.__name__}_params.txt", "a") as f:
            f.write(str(datetime.datetime.now()) + ", " + str(params) + ", " + str(model_results[model_results["Class"]==metric]["Metric"].mean()) + "\n")
        return model_results[model_results["Class"]==metric]["Metric"].mean()

    from pyhopper.callbacks import History
    search = pyhopper.Search(param_grid)

    best_params = search.run(objective, "maximize", time, n_jobs=n_jobs, seeding_ratio=0.5)
    
    with open(f"{DATA}_{model_fn.__name__}_best_params.txt", "a") as f:
            f.write(str(best_params))
    
    print(f"{DATA}_{model_fn.__name__}_{best_params}")
    return best_params, search.history


xgboost_best = 


node_best = 

nn_fn1_best_params = 

nn_fn2_best_params = 

nn_fn3_best_params = 

hp_pca_best_params = 


hp_best_params = 


rf_best = 

# TRAIN MODELS

### XGBoost

In [60]:
from xgboost import XGBClassifier

In [61]:
import xgboost
xgboost.__version__

'1.6.0'

In [62]:
def get_xgboost(**params):
    random_seed = np.random.randint(1024)
    def _inner(**args):
        return XGBClassifier(
            verbosity=0,
            random_state=random_seed,
            use_label_encoder=False,
            **params,
            **args
        )
    return _inner    

#### Hyperparam tuning

In [118]:
param_grid = {
                'n_estimators': pyhopper.choice([50, 100, 250, 500, 1000, 3000]),
                'max_depth': pyhopper.choice([2, 3, 5, 10, 15]),
                'learning_rate': pyhopper.float(1e-5,1e-1, log=True),
                'min_child_weight': pyhopper.choice([1, 2, 4, 8, 16, 32]),
                'gamma': pyhopper.choice([0, 0.001, 0.1, 1]),
             }

xgbt_best1, xgbt_history1 = pyhopper_best_params(get_xgboost, param_grid, time="2260m")

2023-01-05 23:03:20.713 | INFO     | __main__:pyhopper_best_params:7 - pyhopper_best_params get_xgboost


  0%|          | [00:00<?]

Search is scheduled for 1 day 13:40:00 (h:m:s)
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
========================= Summary =========================
Mode              : Best f : Steps : Time                  
----------------  : ----   : ----  : ----                  
Initial solution  : 22.45  : 1     : 03:59 (m:s)           
Random seeding    : 70.4   : 8     : 1 day 04:56:16 (h:m:s)
Local sampling    : 68.57  : 1     : 15:26:25 (h:m:s)

In [63]:
xgbt_best1={'n_estimators': 3000, 'max_depth': 10, 'learning_rate': 0.017180692836176483, 'min_child_weight': 2, 'gamma': 0.001}


In [64]:
param_grid = {
                'subsample': pyhopper.choice([0.5, 0.6, 0.7, 0.8, 0.9, 1]),
                'reg_lambda': pyhopper.float(1e-5, 10, init=0, log=True),
                'reg_alpha': pyhopper.float(1e-5, 10, init=0, log=True),
             }


xgbt_best2, xgbt_history2 = pyhopper_best_params(get_xgboost, param_grid, time=f"{5*60}m", default_params=xgbt_best1)

2023-01-16 18:27:25.983 | INFO     | __main__:pyhopper_best_params:7 - pyhopper_best_params get_xgboost


  0%|          | [00:00<?]

Search is scheduled for 05:00:00 (h:m:s)
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
====================== Summary ======================
Mode              : Best f : Steps : Time            
----------------  : ----   : ----  : ----            
Initial solution  : 64.68  : 1     : 03:24:31 (h:m:s)
----------------  : ----   : ----  : ----            
Total             : 64.68  : 1     : 03:24:31 (h:m:s)
devanagari_get_xgboost_{'subsample': 0.5, 'reg_lambda': 0, 'reg_alpha': 0}


#### Best Params

'Ionosphere' {'gamma': 0, 'learning_rate': 0.1, 'max_depth': 5, 'min_child_weight': 2, 'n_estimators': 1500}


In [65]:
xgboost_best = {**xgbt_best1, **xgbt_best2}

In [73]:
xgboost_best

{'n_estimators': 3000,
 'max_depth': 10,
 'learning_rate': 0.017180692836176483,
 'min_child_weight': 2,
 'gamma': 0.001,
 'subsample': 0.5,
 'reg_lambda': 0,
 'reg_alpha': 0}

In [66]:
data_size = max_size

xgb_dframe = test_model(get_xgboost(**xgboost_best),
                        (X, y),
                        data_size,
                        label_encoder=None, iters=10)

iter 1 of 10
iter 2 of 10
iter 3 of 10
iter 4 of 10
iter 5 of 10
iter 6 of 10
iter 7 of 10
iter 8 of 10
iter 9 of 10
iter 10 of 10


In [67]:
res = xgb_dframe[xgb_dframe["Class"]=="Total"].reset_index(drop=True)["Metric"]
print(f"{data_size}: {res.mean():.2f} ~ {res.std():.2f} (max: {res.max():.2f})")

9038: 73.63 ~ 0.76 (max: 74.55)


In [68]:
xgb_dframe.groupby(['Class']).mean()

,Metric
Class,
0,72.006875
1,71.047065
2,77.601798
3,71.464833
4,68.937070
5,75.742993
6,57.842411
7,73.220518
8,49.920677


In [69]:
xgb_dframe.groupby(['Class']).std()

,Metric
Class,
0,5.481655
1,8.135479
2,3.692846
3,5.781381
4,4.311913
5,6.922080
6,6.146228
7,4.228652
8,8.315695


### NODE

In [70]:
from qhoptim.pyt import QHAdam

In [71]:
def node_fn(layer_dim=128, num_layers=1, depth=3, batch_size=32):
    def _inner():
        network = torch.nn.Sequential(
            node.DenseBlock(X_train.shape[1], 
                            layer_dim=layer_dim,
                            num_layers=num_layers, 
                            tree_dim=n_classes+1, 
                            depth=depth, 
                            flatten_output=False,
                            choice_function=node.entmax15, 
                            bin_function=node.entmoid15
                           ),
            node.Lambda(lambda x: x.mean(dim=1))
        )
        
        
        network = network.to(DEVICE)
        network.device=DEVICE
        
        with torch.no_grad():
            res = network(torch.as_tensor(X_train[:1000], device=DEVICE).to(torch.float32))
        
            
        optimizer_params = { 'nus':(0.7, 1.0), 'betas':(0.95, 0.998) }
        optim = QHAdam(network.parameters(), **optimizer_params)
            
        network = SimpleSklearnInterface(network, device=DEVICE, epochs=150, batch_size=batch_size)
        network.optimizer = optim
        return network
    return _inner

#### Tune hyperparams

In [72]:
# 'layer_dim': hp.quniform('layer_dim', 100, 1200, 100),
# 'num_layers': hp.quniform('num_layers', 1, 4, 1),
# 'depth': hp.quniform('depth', 2, 7, 1)
                    
param_grid = {
    'layer_dim': pyhopper.int(64, 512, power_of=2), #1024
    'num_layers': pyhopper.int(1, 5),
    'depth': pyhopper.int(2, 7),
    'batch_size': pyhopper.choice([128]),
}

node_best, node_history = pyhopper_best_params(node_fn, param_grid, time=f"{4*60}m", n_jobs=1)
node_best

2023-01-17 13:45:31.593 | INFO     | __main__:pyhopper_best_params:7 - pyhopper_best_params node_fn


  0%|          | [00:00<?]

Search is scheduled for 04:00:00 (h:m:s)
iter 1 of 5


/home/z1157095/hypernet-cnn/deps/node/node/lib/odst.py:72: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  bin_codes = (indices.view(1, -1) // offsets.view(-1, 1) % 2).to(torch.float32)
/home/z1157095/anaconda3/envs/hyper-mnist/lib/python3.9/site-packages/qhoptim/pyt/qhadam.py:133: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha) (Triggered internally at  ../torch/csrc/utils/python_arg_parser.cpp:1055.)
  exp_avg.mul_(beta1_adj).add_(1.0 - beta1_adj, d_p)


iter 2 of 5


...

/home/z1157095/hypernet-cnn/deps/node/node/lib/odst.py:72: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  bin_codes = (indices.view(1, -1) // offsets.view(-1, 1) % 2).to(torch.float32)


iter 1 of 5


/home/z1157095/hypernet-cnn/deps/node/node/lib/odst.py:72: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  bin_codes = (indices.view(1, -1) // offsets.view(-1, 1) % 2).to(torch.float32)


Remote process caught exception in objective function: 
Traceback (most recent call last):
  File "/home/z1157095/anaconda3/envs/hyper-mnist/lib/python3.9/site-packages/pyhopper/parallel.py", line 223, in execute
    iter_or_result = objective_function(candidate, **kwargs)
  File "/tmp/ipykernel_891768/1403394827.py", line 12, in objective
    model_results = test_model(
  File "/tmp/ipykernel_891768/3861398633.py", line 60, in test_model
    model = model_fn()
  File "/tmp/ipykernel_891768/2870764771.py", line 21, in _inner
    res = network(torch.as_tensor(X_train[:1000], device=DEVICE).to(torch.float32))
  File "/home/z1157095/anaconda3/envs/hyper-mnist/lib/python3.9/site-packages/torch/nn/modules/module.py", line 1110, in _call_impl
    return forward_call(*input, **kwargs)
  File "/home/z1157095/anaconda3/envs/hyper-mnist/lib/python3.9/site-packages/torch/nn/modules/container.py", line 141, in forward
    input = module(input)
  File "/home/z1157095/anaconda3/envs/hyper-mnist/lib/

ValueError: Pyhopper - Remote process caught exception

#### Use best hyperparams

In [ ]:
%%capture --no-stdout

size=max_size

node_results = test_model(node_fn(**node_best),
                    (X, y),
                    size,
                    label_encoder=None, iters=10)
res = node_results[node_results["Class"]=="roc_auc"].reset_index(drop=True)["Metric"]
print(f"{size}: {res.mean():.2f} ~ {res.std():.2f}, (max: {res.max():.2f})")

### Dropout Neural network

In [ ]:
def network_fn1(epochs=100, drop1=0.3, drop2=0.5, batch_size=32, lr=3e-4):
    
    def _inner():
        network = torch.nn.Sequential(
                        torch.nn.Dropout(drop1),
                        torch.nn.Linear(n_features, 64),
                        torch.nn.ReLU(),
                        torch.nn.Dropout(drop2),
                        torch.nn.Linear(64, n_classes)
                    ).to(DEVICE).train()

        network = SimpleSklearnInterface(network, epochs=epochs, batch_size=batch_size, lr=lr)
        return network
    return _inner



def network_fn2(epochs=100, drop1=0.3, drop2=0.5, drop3=0.5, batch_size=32, lr=3e-4):
    
    def _inner():
        network = torch.nn.Sequential(
                        torch.nn.Dropout(drop1),
                        torch.nn.Linear(n_features, 64),
                        torch.nn.ReLU(),
                        torch.nn.Dropout(drop2),
                        torch.nn.Linear(64, 64),
                        torch.nn.ReLU(),
                        torch.nn.Dropout(drop3),
                        torch.nn.Linear(64, n_classes)
                    ).to(DEVICE).train()

        network = SimpleSklearnInterface(network, epochs=epochs, batch_size=batch_size, lr=lr)
        return network
    return _inner



def network_fn3(epochs=100, drop1=0.3, drop2=0.5, drop3=0.5, drop4=0.5, batch_size=32, lr=3e-4):
    
    def _inner():
        network = torch.nn.Sequential(
                        torch.nn.Dropout(drop1),
                        torch.nn.Linear(n_features, 64),
                        torch.nn.ReLU(),
                        torch.nn.Dropout(drop2),
                        torch.nn.Linear(64, 128),
                        torch.nn.ReLU(),
                        torch.nn.Dropout(drop3),
                        torch.nn.Linear(128, 64),
                        torch.nn.ReLU(),
                        torch.nn.Dropout(drop4),
                        torch.nn.Linear(64, n_classes)
                    ).to(DEVICE).train()

        network = SimpleSklearnInterface(network, epochs=epochs, batch_size=batch_size, lr=lr)
        return network
    return _inner

#### Find Hyperparams

### Dropout 1 layer

In [47]:
param_grid = {
                "epochs": pyhopper.choice([100, 150]),
                "drop1": pyhopper.choice([0.1, 0.3, 0.5, 0.7], is_ordinal=True),
                "drop2": pyhopper.choice([0.1, 0.3, 0.5, 0.7], is_ordinal=True),
                "lr": pyhopper.choice([3e-5, 3e-4, 3e-3, 3e-2]),
                "batch_size": pyhopper.choice([128]),
             }

nn_fn1_best_params, nn_fn1_history = pyhopper_best_params(network_fn1, param_grid, time="60m")
nn_fn1_best_params

2023-01-03 09:21:40.529 | INFO     | __main__:pyhopper_best_params:7 - pyhopper_best_params network_fn1


  0%|          | [00:00<?]

Search is scheduled for 01:00:00 (h:m:s)
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
==================== Summary ===================
Mode              : Best f : Steps : Time       
----------------  : -

{'epochs': 150, 'drop1': 0.7, 'drop2': 0.1, 'lr': 0.003, 'batch_size': 128}

In [48]:
data_size = max_size

nn1_results = test_model(network_fn1(**nn_fn1_best_params),
                (X, y),
                data_size,
                None, 10)

iter 1 of 10
iter 2 of 10
iter 3 of 10
iter 4 of 10
iter 5 of 10
iter 6 of 10
iter 7 of 10
iter 8 of 10
iter 9 of 10
iter 10 of 10


In [49]:
res = nn1_results[nn1_results["Class"]=="Balanced Acc score"]["Metric"]
print(f"{data_size}: {res.mean():.2f} ~ {res.std():.2f} (max: {res.max():.2f})")

9038: nan ~ nan (max: nan)


### Dropout 2 layers

In [50]:
param_grid = {
                "epochs": pyhopper.choice([100, 150]),
                "drop1": pyhopper.choice([0.1, 0.3, 0.5, 0.7], is_ordinal=True),
                "drop2": pyhopper.choice([0.1, 0.3, 0.5, 0.7], is_ordinal=True),
                "drop3": pyhopper.choice([0.1, 0.3, 0.5, 0.7], is_ordinal=True),
                "lr": pyhopper.choice([3e-5, 3e-4, 3e-3, 3e-2]),
                "batch_size": pyhopper.choice([128]),
             }

nn_fn2_best_params, nn_fn2_history = pyhopper_best_params(network_fn2, param_grid, time="70m")
nn_fn2_best_params

2023-01-03 10:29:38.308 | INFO     | __main__:pyhopper_best_params:7 - pyhopper_best_params network_fn2


  0%|          | [00:00<?]

Search is scheduled for 01:10:00 (h:m:s)
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5

{'epochs': 150,
 'drop1': 0.3,
 'drop2': 0.1,
 'drop3': 0.1,
 'lr': 0.003,
 'batch_size': 128}

In [51]:
data_size = max_size

nn2_results = test_model(network_fn2(**nn_fn2_best_params),
                (X, y),
                data_size,
                None, 10)

iter 1 of 10
iter 2 of 10
iter 3 of 10
iter 4 of 10
iter 5 of 10
iter 6 of 10
iter 7 of 10
iter 8 of 10
iter 9 of 10
iter 10 of 10


### Dropout 3 layers

In [52]:
param_grid = {
                "epochs": pyhopper.choice([100, 150]),
                "drop1": pyhopper.choice([0.1, 0.3, 0.5, 0.7], is_ordinal=True),
                "drop2": pyhopper.choice([0.1, 0.3, 0.5, 0.7], is_ordinal=True),
                "drop3": pyhopper.choice([0.1, 0.3, 0.5, 0.7], is_ordinal=True),
                "drop4": pyhopper.choice([0.1, 0.3, 0.5, 0.7], is_ordinal=True),
                "lr": pyhopper.choice([3e-5, 3e-4, 3e-3, 3e-2]),
                "batch_size": pyhopper.choice([128]),
             }

nn_fn3_best_params, nn_fn3_history = pyhopper_best_params(network_fn3, param_grid, time="75m")
nn_fn3_best_params

2023-01-03 11:48:55.434 | INFO     | __main__:pyhopper_best_params:7 - pyhopper_best_params network_fn3


  0%|          | [00:00<?]

Search is scheduled for 01:15:00 (h:m:s)
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5

{'epochs': 150,
 'drop1': 0.1,
 'drop2': 0.1,
 'drop3': 0.1,
 'drop4': 0.1,
 'lr': 0.0003,
 'batch_size': 128}

In [53]:
data_size = max_size

nn3_results = test_model(network_fn3(**nn_fn3_best_params),
                (X, y),
                data_size,
                None, 10)

iter 1 of 10
iter 2 of 10
iter 3 of 10
iter 4 of 10
iter 5 of 10
iter 6 of 10
iter 7 of 10
iter 8 of 10
iter 9 of 10
iter 10 of 10


### Mask sizes

In [54]:
mask_sizes = [int(n_features * 0.3), int(n_features * 0.5), int(n_features * 0.7), int(n_features * 0.8)]
mask_sizes

[235, 392, 548, 627]

### HypernetworkPCA

In [55]:
def network_pca_fn(epochs=100, masks_no=100, mask_size=100, target_size=100, n_comp=5, lr=3e-4, batch_size=64, verbose=False):
    def _inner():
        hypernet = HypernetworkPCA(
                        target_architecture=[(mask_size, target_size), (target_size, n_classes)], 
                        test_nodes=masks_no,
                        architecture=torch.nn.Sequential(torch.nn.Linear(n_comp, 64), 
                            torch.nn.ReLU(),
                            torch.nn.Linear(64, 128),
                            torch.nn.ReLU(),
                            torch.nn.Dropout(),
                            torch.nn.Linear(128, 128),
                            torch.nn.ReLU(),
                        ),
                        mode=TrainingModes.CARTHESIAN,
                        input_size=n_features
                    ).to(DEVICE)    
        hypernet = hypernet.train()

        network = HypernetworkSklearnInterface(hypernet, device=DEVICE, epochs=epochs, batch_size=batch_size, verbose=verbose, lr=lr)
        return network
    return _inner

In [56]:
def figure_to_html():
    tmpfile = BytesIO()
    plt.gcf().savefig(tmpfile, format='png')
    encoded = base64.b64encode(tmpfile.getvalue()).decode('utf-8')

    html = '<img src=\'data:image/png;base64,{}\'>'.format(encoded)
    return html

#### Find hyperparams

In [57]:
param_grid = {
    "epochs": pyhopper.choice([100]),
    "masks_no": pyhopper.choice([5, 7, 20]),
    "mask_size": pyhopper.choice(mask_sizes),
    "target_size": pyhopper.choice([5, 10, 20, 50]),
    "n_comp": pyhopper.choice([5]),
    "lr": pyhopper.choice([3e-5, 3e-4, 3e-3, 3e-2, 3e-1]),
    "batch_size": pyhopper.choice([128]),

}

hp_pca_best_params, hp_pca_history = pyhopper_best_params(network_pca_fn, param_grid, time="480m")
hp_pca_best_params

2023-01-03 13:13:00.770 | INFO     | __main__:pyhopper_best_params:7 - pyhopper_best_params network_pca_fn


  0%|          | [00:00<?]

Search is scheduled for 08:00:00 (h:m:s)
iter 1 of 5
torch.Size([1, 128])
784
iter 2 of 5
torch.Size([1, 128])
784
iter 3 of 5
torch.Size([1, 128])
784
iter 4 of 5
torch.Size([1, 128])
784
iter 5 of 5
torch.Size([1, 128])
784
iter 1 of 5
torch.Size([1, 128])
784
iter 2 of 5
torch.Size([1, 128])
784
iter 3 of 5
torch.Size([1, 128])
784
iter 4 of 5
torch.Size([1, 128])
784
iter 5 of 5
torch.Size([1, 128])
784
iter 1 of 5
torch.Size([1, 128])
784
iter 2 of 5
torch.Size([1, 128])
784
iter 3 of 5
torch.Size([1, 128])
784
iter 4 of 5
torch.Size([1, 128])
784
iter 5 of 5
torch.Size([1, 128])
784
iter 1 of 5
torch.Size([1, 128])
784
iter 2 of 5
torch.Size([1, 128])
784
iter 3 of 5
torch.Size([1, 128])
784
iter 4 of 5
torch.Size([1, 128])
784
iter 5 of 5
torch.Size([1, 128])
784
iter 1 of 5
torch.Size([1, 128])
784
iter 2 of 5
torch.Size([1, 128])
784
iter 3 of 5
torch.Size([1, 128])
784
iter 4 of 5
torch.Size([1, 128])
784
iter 5 of 5
torch.Size([1, 128])
784
iter 1 of 5
torch.Size([1, 128])
7

{'epochs': 100,
 'masks_no': 7,
 'mask_size': 235,
 'target_size': 50,
 'n_comp': 5,
 'lr': 0.0003,
 'batch_size': 128}

In [58]:
hp_pca_best_params

{'epochs': 100,
 'masks_no': 7,
 'mask_size': 235,
 'target_size': 50,
 'n_comp': 5,
 'lr': 0.0003,
 'batch_size': 128}

'Libras'
{'epochs': 150,
 'masks_no': 70,
 'mask_size': 20,
 'target_size': 10,
 'n_comp': 10}

'Lymphography' {'epochs': 120, 'masks_no': 50, 'mask_size': 4, 'target_size': 20, 'n_comp': 6}

In [59]:
DATA

'devanagari'

Ionosphere
{'epochs': 100, 'masks_no': 60, 'mask_size': 5, 'target_size': 10}


In [60]:
epochs = 100#hp_pca_best_params['epochs']
masks_no = hp_pca_best_params['masks_no']
mask_size = hp_pca_best_params['mask_size']
target_size = hp_pca_best_params['target_size']
n_comp = hp_pca_best_params['n_comp']
lr = hp_pca_best_params['lr']
batch_size = hp_pca_best_params['batch_size']
data_size = max_size

nn_pca_results = test_model(network_pca_fn(target_size=target_size, mask_size=mask_size, masks_no=masks_no, n_comp=n_comp, epochs=epochs, verbose=True),
                (X, y),
                data_size,
                None, 10)

# exp.log_table("metrics.csv", nn_pca_results.groupby("Class").mean())
# exp.log_metric("f1_score", nn_pca_results.groupby("Class").mean().loc["F1 score"])

iter 1 of 10
torch.Size([1, 128])
784


100%|██████████| 100/100 [05:01<00:00,  3.01s/it]


iter 2 of 10
torch.Size([1, 128])
784


100%|██████████| 100/100 [04:54<00:00,  2.94s/it]


iter 3 of 10
torch.Size([1, 128])
784


...

100%|██████████| 100/100 [04:53<00:00,  2.94s/it]


iter 9 of 10
torch.Size([1, 128])
784


100%|██████████| 100/100 [04:52<00:00,  2.92s/it]


iter 10 of 10
torch.Size([1, 128])
784


100%|██████████| 100/100 [04:56<00:00,  2.96s/it]


In [61]:
res = nn_pca_results[nn_pca_results["Class"]=="Total"].reset_index(drop=True)["Metric"]
print(f"{data_size}: {res.mean():.2f} ~ {res.std():.2f} (max: {res.max():.2f})")

9038: 78.67 ~ 0.65 (max: 79.58)


### Hypernetwork

In [62]:
def network_hp_fn(epochs=150, masks_no=100, mask_size=100, target_size=100, lr=3e-4, batch_size=64, verbose=False):
    def _inner():
        hypernet = Hypernetwork(
                        target_architecture=[(mask_size, target_size), (target_size, n_classes)],
                        test_nodes=masks_no,
                        architecture=torch.nn.Sequential(torch.nn.Linear(n_features, 64), 
                            torch.nn.ReLU(),
                            torch.nn.Linear(64, 128),
                            torch.nn.ReLU(),
                            torch.nn.Dropout(),
                            torch.nn.Linear(128, 128),
                            torch.nn.ReLU(),
                        ),
                        mode=TrainingModes.CARTHESIAN,
                    ).to(DEVICE)    
        hypernet = hypernet.train()

        network = HypernetworkSklearnInterface(hypernet, device=DEVICE, epochs=epochs, batch_size=batch_size, verbose=verbose, lr=lr)
        return network
    return _inner


#### Find hyperparams

In [63]:
param_grid = {
    "epochs": pyhopper.choice([100]),
    "masks_no": pyhopper.choice([3, 5, 7, 20]),
    "mask_size": pyhopper.choice(mask_sizes),
    "target_size": pyhopper.choice([5, 10, 20, 50]),
    "lr": pyhopper.choice([3e-5, 3e-4, 3e-3, 3e-2, 3e-1]),
    "batch_size": pyhopper.choice([128]),
}

hp_best_params, hp_history = pyhopper_best_params(network_hp_fn, param_grid, time="480m")
hp_best_params

2023-01-03 22:19:35.828 | INFO     | __main__:pyhopper_best_params:7 - pyhopper_best_params network_hp_fn


  0%|          | [00:00<?]

Search is scheduled for 08:00:00 (h:m:s)
iter 1 of 5
torch.Size([1, 128])
iter 2 of 5
torch.Size([1, 128])
iter 3 of 5
torch.Size([1, 128])
iter 4 of 5
torch.Size([1, 128])
iter 5 of 5
torch.Size([1, 128])
iter 1 of 5
torch.Size([1, 128])
iter 2 of 5
torch.Size([1, 128])
iter 3 of 5
torch.Size([1, 128])
iter 4 of 5
torch.Size([1, 128])
iter 5 of 5
torch.Size([1, 128])
iter 1 of 5
torch.Size([1, 128])
iter 2 of 5
torch.Size([1, 128])
iter 3 of 5
torch.Size([1, 128])
iter 4 of 5
torch.Size([1, 128])
iter 5 of 5
torch.Size([1, 128])
iter 1 of 5
torch.Size([1, 128])
iter 2 of 5
torch.Size([1, 128])
iter 3 of 5
torch.Size([1, 128])
iter 4 of 5
torch.Size([1, 128])
iter 5 of 5
torch.Size([1, 128])
iter 1 of 5
torch.Size([1, 128])
iter 2 of 5
torch.Size([1, 128])
iter 3 of 5
torch.Size([1, 128])
iter 4 of 5
torch.Size([1, 128])
iter 5 of 5
torch.Size([1, 128])
iter 1 of 5
torch.Size([1, 128])
iter 2 of 5
torch.Size([1, 128])
iter 3 of 5
torch.Size([1, 128])
iter 4 of 5
torch.Size([1, 128])
it

{'epochs': 100,
 'masks_no': 20,
 'mask_size': 235,
 'target_size': 50,
 'lr': 0.0003,
 'batch_size': 128}

In [64]:
hp_best_params

{'epochs': 100,
 'masks_no': 20,
 'mask_size': 235,
 'target_size': 50,
 'lr': 0.0003,
 'batch_size': 128}

#### Train using the best hyperparams

In [65]:
epochs = 100#hp_best_params['epochs']
masks_no = hp_best_params['masks_no']
mask_size = hp_best_params['mask_size']
target_size = hp_best_params['target_size']
data_size = max_size
batch_size = hp_best_params['batch_size']
lr = hp_best_params['lr']


hyper_results = test_model(network_hp_fn(epochs, masks_no, mask_size, target_size, batch_size=batch_size,verbose=True),
                    (X, y),
                    data_size,
                    None, 10)

iter 1 of 10
torch.Size([1, 128])


100%|██████████| 100/100 [07:19<00:00,  4.39s/it]


iter 2 of 10
torch.Size([1, 128])


100%|██████████| 100/100 [07:10<00:00,  4.30s/it]


iter 3 of 10
torch.Size([1, 128])


...

100%|██████████| 100/100 [07:13<00:00,  4.33s/it]


iter 9 of 10
torch.Size([1, 128])


100%|██████████| 100/100 [07:11<00:00,  4.31s/it]


iter 10 of 10
torch.Size([1, 128])


100%|██████████| 100/100 [07:18<00:00,  4.39s/it]


In [66]:
res = hyper_results[hyper_results["Class"]=="balanced_accuracy"].reset_index(drop=True)["Metric"]
print(f"{data_size}: {res.mean():.2f} ~ {res.std():.2f}, (max: {res.max():.2f})")

9038: 78.92 ~ 0.66, (max: 79.78)


### Random forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
def get_rf(**params):
    random_seed = np.random.randint(1024)
    def _inner():
        return RandomForestClassifier(
            random_state=random_seed,
            **params
        )
    return _inner

#### Find hyperparams

In [ ]:
param_grid = {
                'n_estimators': pyhopper.choice([50, 100, 200, 500, 1000, 3000]),
                'max_features': pyhopper.choice([None, 'sqrt', 0.2, 0.3, 0.5, 0.7]),
                'criterion' : pyhopper.choice(['gini', 'entropy']),
                'max_depth': pyhopper.choice([None, 2, 4, 8, 16]),
             }

rf_best, rf_history = pyhopper_best_params(get_rf, param_grid, time=f"{5*60}m")
rf_best

#### Use best params

In [ ]:
size = max_size

rf_dframe = test_model(get_rf(**rf_best), 
                        (X, y),
                        size,
                        None, iters=10)

# Collect analytics

In [ ]:
d = {}

In [ ]:
d['Random forest'] = rf_dframe.groupby("Class")['Metric'].agg(['mean', 'std', 'max'])
d['Hypernet'] = hyper_results.groupby("Class")['Metric'].agg(['mean', 'std', 'max'])
d['HypernetPCA'] = nn_pca_results.groupby("Class")['Metric'].agg(['mean', 'std', 'max'])
d['Dropout_1'] = nn1_results.groupby("Class")['Metric'].agg(['mean', 'std', 'max'])
d['Dropout_2'] = nn2_results.groupby("Class")['Metric'].agg(['mean', 'std', 'max'])
d['Dropout_3'] = nn3_results.groupby("Class")['Metric'].agg(['mean', 'std', 'max'])
d['Node'] = node_results.groupby("Class")['Metric'].agg(['mean', 'std', 'max'])
d['XGBoost'] = xgb_dframe.groupby("Class")['Metric'].agg(['mean', 'std', 'max'])

In [126]:
all_models_df=pd.concat(d, axis=0)
all_models_df

mean       std        max
              Class                                            
Random forest 0                  71.240085  5.714773  80.645161
              1                  68.746695  4.649648  77.049180
              2                  83.968800  4.256506  88.709677
              3                  78.677948  5.617256  87.096774
              4                  79.190904  4.150279  85.483871
...                                    ...       ...        ...
XGBoost       55                 91.511628  4.103771  96.511628
              56                 93.488372  1.662605  95.348837
              57                 89.186047  3.719719  95.348837
              Total              73.257615  0.540365  74.445018
              balanced_accuracy  72.029508  0.575351  73.262072

[480 rows x 3 columns]

In [127]:
os.environ['COMET_KEY']

'UXrV5UxyhTK3cyQNG6BDuc4bE'

In [128]:
all_models_df.to_csv(f"{DATA}_metrics.csv")

In [129]:
exp = Experiment(os.environ.get("COMET_KEY"), 'hypernet-uci-tune')
# exp.log_parameters({"epochs": epochs, "mask_size": mask_size, "masks_no": masks_no, "data_size": data_size})
exp.add_tag(f"hypernet-tune2{DATA}")
exp.log_table(f"{DATA}_metrics.csv", all_models_df)

COMET WARNING: Comet has disabled auto-logging functionality as it has been imported after the following ML modules: torch. Metrics and hyperparameters can still be logged using comet_ml.log_metrics() and comet_ml.log_parameters()
COMET INFO: ---------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.com/abulenok/hypernet-uci-tune/19a6093dbd6748c89a5a3323f19ad0d0
COMET INFO:   Parameters:
COMET INFO:     begin_iteration  : 0
COMET INFO:     copy             : True
COMET INFO:     end_iteration    : 2999
COMET INFO:     gamma            : 0.001
COMET INFO:     learning_rate    : 0.0011
COMET INFO:     max_depth        : 20
COMET INFO:     min_child_weight : 32
COMET INFO:     num_class        : 58
COMET INFO:     objective        : multi:softprob
COMET INFO:     random_state     : 691
COMET INFO:     rank             : 0

{'web': 'https://www.comet.com/api/asset/download?assetId=5697c0c1f1674068b8d69e723ac12b2a&experimentKey=ac01f58168f840a0903f30ed010ddd3e',
 'api': 'https://www.comet.com/api/rest/v2/experiment/asset/get-asset?assetId=5697c0c1f1674068b8d69e723ac12b2a&experimentKey=ac01f58168f840a0903f30ed010ddd3e',
 'assetId': '5697c0c1f1674068b8d69e723ac12b2a'}

### Replace some data in existing

In [130]:
# tmp_df = pd.concat(d, axis=0)
# tmp_df = tmp_df.reset_index()
# tmp_df = tmp_df.rename(columns={tmp_df.columns[0]: DATA})

# tmp_df

In [131]:
# all_models_df = pd.read_csv(f"{DATA}_metrics.csv")
# all_models_df = all_models_df.rename(columns={all_models_df.columns[0]: DATA})
# all_models_df = all_models_df.drop(all_models_df[all_models_df.iloc[:, 0] == 'Hypernet'].index)
# all_models_df = all_models_df.drop(all_models_df[all_models_df.iloc[:, 0] == 'HypernetPCA'].index)

# all_models_df

In [132]:
# all_models_df = pd.concat([all_models_df, tmp_df])
# all_models_df

In [133]:
f"{DATA}_metrics.csv"

'devanagari_metrics.csv'

In [134]:
all_models_df = pd.read_csv(f"{DATA}_metrics.csv")

In [135]:
all_models_df.reset_index()

,index,Unnamed: 0,Class,mean,std,max
0,0,Random forest,0,71.240085,5.714773,80.645161
1,1,Random forest,1,68.746695,4.649648,77.049180
2,2,Random forest,2,83.968800,4.256506,88.709677
3,3,Random forest,3,78.677948,5.617256,87.096774
4,4,Random forest,4,79.190904,4.150279,85.483871
...,...,...,...,...,...,...
475,475,XGBoost,55,91.511628,4.103771,96.511628
476,476,XGBoost,56,93.488372,1.662605,95.348837
477,477,XGBoost,57,89.186047,3.719719,95.348837
478,478,XGBoost,Total,73.257615,0.540365,74.445018


In [136]:
# all_models_df = all_models_df.drop(all_models_df.columns[0], axis=1)
# all_models_df

In [137]:
all_models_df[all_models_df['Class'] == 'balanced_accuracy']

,Unnamed: 0,Class,mean,std,max
59,Random forest,balanced_accuracy,71.145274,0.725982,72.273741
119,Hypernet,balanced_accuracy,78.919606,0.663281,79.777526
179,HypernetPCA,balanced_accuracy,77.668715,0.679971,78.596315
239,Dropout_1,balanced_accuracy,77.272556,0.928859,78.591582
299,Dropout_2,balanced_accuracy,76.048004,0.954580,77.538374
359,Dropout_3,balanced_accuracy,75.243723,0.468235,75.794244
419,Node,balanced_accuracy,78.202939,1.076930,79.530906
479,XGBoost,balanced_accuracy,72.029508,0.575351,73.262072


In [138]:
tmp = all_models_df

In [139]:
tmp = tmp.rename(columns={tmp.columns[0]: DATA})

In [140]:
tmp

,devanagari,Class,mean,std,max
0,Random forest,0,71.240085,5.714773,80.645161
1,Random forest,1,68.746695,4.649648,77.049180
2,Random forest,2,83.968800,4.256506,88.709677
3,Random forest,3,78.677948,5.617256,87.096774
4,Random forest,4,79.190904,4.150279,85.483871
...,...,...,...,...,...
475,XGBoost,55,91.511628,4.103771,96.511628
476,XGBoost,56,93.488372,1.662605,95.348837
477,XGBoost,57,89.186047,3.719719,95.348837
478,XGBoost,Total,73.257615,0.540365,74.445018


In [141]:
tmp[tmp['Class'] == "balanced_accuracy"]

,devanagari,Class,mean,std,max
59,Random forest,balanced_accuracy,71.145274,0.725982,72.273741
119,Hypernet,balanced_accuracy,78.919606,0.663281,79.777526
179,HypernetPCA,balanced_accuracy,77.668715,0.679971,78.596315
239,Dropout_1,balanced_accuracy,77.272556,0.928859,78.591582
299,Dropout_2,balanced_accuracy,76.048004,0.954580,77.538374
359,Dropout_3,balanced_accuracy,75.243723,0.468235,75.794244
419,Node,balanced_accuracy,78.202939,1.076930,79.530906
479,XGBoost,balanced_accuracy,72.029508,0.575351,73.262072
